In [17]:
# importation des modules
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [18]:
# chargement des sous-corpus
data_spon = pd.read_csv("../data/corpus_spon.csv")
data_prep = pd.read_csv("../data/corpus_prep.csv")

In [19]:
# corpus avec questions uniquement
data_spon_q = data_spon["question"]
data_prep_q = data_prep["question"]

In [20]:
# tf-idf
from nltk.corpus import stopwords
stopwords_fr = stopwords.words('french')
vectorizer = TfidfVectorizer(stop_words=stopwords_fr)
X_prep = vectorizer.fit_transform(data_prep_q)
X_spon = vectorizer.fit_transform(data_spon_q)

Problème après la vectorisation : réduction de dimensions. 
\
Plusieurs méthodes possibles :
- PCA (Principal Component Analysis)
- SVD (Truncated SVD)

PCA ne marche visiblement pas sur nos données (*too sparse?*).

In [58]:
# méthodes de réduction de dimensions
# PCA
from sklearn.decomposition import PCA
pca = PCA()
pca_fit = pca.fit(X)

#SVD
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np

X = csr_matrix(X)
print(X.shape)
svd = TruncatedSVD(n_components=400, n_iter=7, random_state=42)
svd.fit(X)
fit_trans = svd.fit_transform(X) # fit + réduction de dimension

(1002, 634)


TruncatedSVD(n_components=400, n_iter=7, random_state=42)

In [35]:
# création du modèle kmeans
true_k = 10
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init='auto', algorithm="elkan")
# on fit le modèle sur chacun des sous-copus
model.fit(X_prep) 

KMeans(algorithm='elkan', max_iter=200, n_clusters=10, n_init='auto')

In [36]:
# affichage des "top termes" dans chaque cluster

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
# faire un dico avec clusters + tops mots
# sauvegarde des résultats dans un fichier txt
with open("kmeans_cluster_prep_cluster_10.txt", "w") as f:
    f.write("Top terms per cluster:")
    for i in range(true_k):
        f.write("Cluster %d: \n" % i)
        for ind in order_centroids[i, :10]:
            f.write(' %s' % terms[ind])
            f.write("\n")
        f.write("\n")
f.close()

In [37]:
# création du modèle kmeans
true_k = 10
model = KMeans(n_clusters=true_k, init='k-means++',
               max_iter=200, n_init='auto', algorithm="elkan")
# on fit le modèle sur chacun des sous-copus
model.fit(X_spon)
# affichage des "top termes" dans chaque cluster
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()


In [38]:
# sauvegarde des résultats dans un fichier txt
with open("kmeans_cluster_spon_cluster_10.txt", "w") as f:
    f.write("Top terms per cluster:")

    for i in range(true_k):
        f.write("Cluster %d: \n" % i)
        for ind in order_centroids[i, :10]:
            f.write(' %s' % terms[ind])
            f.write("\n")
        f.write("\n")
f.close()

In [71]:
print("\n")
print("Prediction")

Y = vectorizer.transform(["où"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["pourquoi"])
prediction = model.predict(Y)
print(prediction)



Prediction
[0]
[7]
